In [1]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
import pyodbc
import logging
import numpy as np

In [77]:
from sqlalchemy import create_engine, event
import urllib
from sqlalchemy.types import Integer, String, Unicode, Boolean, DateTime, Float, Numeric

In [3]:
def save_bigquery(df, dataset, table):
    """Import a csv file into BigQuery"""
    logging.info(table)
    client = bigquery.Client()
    
    table_ref = client.dataset(dataset).table(table)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.autodetect = True
    load_job = client.load_table_from_dataframe(df,
                                                table_ref,
                                                job_config=job_config)
    # Waits for table load to complete.
    load_job.result()

In [4]:
def connect(request):
    server = '10.34.1.4' 
    database = 'ODS_UNILEVER' 
    username = 'BIGQUERY_BI' 
    password = 'AN2 6optimist hurts a backpack' 
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password, timeout=login_timeout)
    cursor = cnxn.cursor()
    tableorigem = 'Pedido'
    sql = 'SELECT * FROM ODS_UNILEVER.infrashop.' + tableorigem
    table = tableorigem
    dataset = "infrashop_raw"
    df = pd.read_sql(sql,cnxn)
    
    logging.info(f'leu tabela = {table}')
    save_bigquery(df, dataset, table)
    logging.info(f'gravou tabela = {table}')


In [13]:
server = '10.34.1.4' 
database = 'ODS_UNILEVER' 
username = 'BIGQUERY_BI' 
password = 'AN2 6optimist hurts a backpack' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
tableorigem = 'Pedido'
sql = 'SELECT * FROM ODS_UNILEVER.infrashop.' + tableorigem
table = tableorigem
dataset = "infrashop_raw"
df = pd.read_sql(sql,cnxn)

In [16]:
df.head()

PedidoId  MasterPedidoId  ClienteId  EnderecoEntregaId  CupomDescontoId  \
0         1               1       4370               4370                0   
1  10519134        10519137      11213              11213                0   
2  10521120        10521123      17990              17990                0   
3  10547235        10547238      20644              20644                0   
4  10548078        10548081      10907              10907                0   

   ValorTotal  ValorTotalFrete  ValorCupomDesconto  ValorCupomDescontoFrete  \
0      333.60              0.0                 0.0                      0.0   
1     3853.36              0.0                 NaN                      NaN   
2      513.82              0.0                 NaN                      NaN   
3      439.07              0.0                 NaN                      NaN   
4      723.88              0.0                 NaN                      NaN   

                IP  ... SistemaOperacional PrazoCd  Marketplace  Fingerprint  \
0  170.238.117.25,  ...            Desktop       0            1   3711524516   
1   186.225.84.192  ...               None       0            0         None   
2    177.37.142.92  ...               None       0            0         None   
3     201.20.109.2  ...               None       0            0         None   
4    170.83.67.251  ...               None       0            0         None   

   SegmentoClienteId                                      HASH  \
0               18.0  03ac23b369cd48a89d2ab3dca60161db3060960a   
1                NaN  fa60fe0848b69c1aa3ad6c66e0704b59ca876d33   
2                NaN  547c8f563363e83cf214974428bed28d7b5f6dd1   
3                NaN  e1e6bcc3c18186c147aab424c9793ccce81cc342   
4                NaN  92477d9b751f33d3236b34e959f42b83e9cdf11c   

                DAT_CARGA  PrazoEntregaTexto  CodigoPoliticaPreco  Migracao  
0 2020-02-29 19:44:20.327               None                 None       NaN  
1 2020-03-04 05:54:47.560               None                 None       1.0  
2 2020-03-04 05:54:47.560               None                 None       1.0  
3 2020-03-04 05:54:47.560               None                 None       1.0  
4 2020-03-04 05:54:47.560               None                 None       1.0  

[5 rows x 33 columns]

In [3]:
server = '10.34.1.4' 
database = 'AMB_BIGQUERY' 
username = 'BIGQUERY_BI' 
password = 'AN2 6optimist hurts a backpack' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [28]:
for index,row in df.iterrows():
    #sql = "INSERT INTO Teste_BigQuery([PedidoId],[MasterPedidoId],[ClienteId]) values (?, ?, ?)"
    cursor.execute("INSERT INTO infrashop.Teste_BigQuery([PedidoId],[MasterPedidoId],[ClienteId]) values (?, ?, ?)", 1, 2, 3)

ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'infrashop.Teste_BigQuery'. (208) (SQLExecDirectW)")

In [37]:
cursor.execute("CREATE TABLE BI_UNILEVER.dw.NewTable ( Column1 varchar(100) NULL ) GO")


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near 'GO'. (102) (SQLExecDirectW)")

In [32]:
df.to_sql(name = 'Teste_BigQuery2', con = cnxn, if_exists = 'append', index = False)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sqlite_master'. (208) (SQLExecDirectW)")

In [4]:
db_params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

In [28]:
df2.head()

SK_PEDIDO  SK_FRONT  SK_PRODUTO  SK_KIT  SK_DATA  SK_CANAL_VENDA  \
0    2234368         1          -2      -2       -2              -2   
1    2234368         1          -2      -2       -2              -2   
2    2234368         1          -2      -2       -2              -2   
3    2234368         1          -2      -2       -2              -2   
4    2234368         1          -2      -2       -2              -2   

   SK_ENDERECO  SK_CLIENTE  SK_DISTRIBUIDOR  SK_TIPO_PAGAMENTO  ...  \
0           -2       15497               -2                 -2  ...   
1           -2       15497               -2                 -2  ...   
2           -2       15497               -2                 -2  ...   
3           -2       15497               -2                 -2  ...   
4           -2       15497               -2                 -2  ...   

   QTD_ITEM_A_ITEM_PS  QTD_ITEM_A_ITEM_PSK  VLR_ITEM_A_ITEM_PS  \
0                -2.0                 -2.0                -2.0   
1                -2.0                 -2.0                -2.0   
2                -2.0                 -2.0                -2.0   
3                -2.0                 -2.0                -2.0   
4                -2.0                 -2.0                -2.0   

   VLR_ITEM_A_ITEM_PSK  FATOR_QTD_PS  FATOR_QTD_PSK  FATOR_BONIFICADO_PS  \
0                 -2.0             0              0                    0   
1                 -2.0             0              0                    0   
2                 -2.0             0              0                    0   
3                 -2.0             0              0                    0   
4                 -2.0             0              0                    0   

   FATOR_BONIFICADO_PSK  DT_INS  FLG_ORIGEM  
0                     0       0           3  
1                     0       0           3  
2                     0       0           3  
3                     0       0           3  
4                     0       0           3  

[5 rows x 24 columns]

In [12]:
df1 = df1.fillna(-2)

In [13]:
df1.to_sql('FAT_PEDIDO_FATURADO',con=engine,index=False,if_exists="append",schema="dbo",
           dtype={   "CD_NOTA_FISCAL": Integer(),
                     "QTD_ITEM_NF": Integer(),
                     "VLR_ITEM_NF": Numeric(10,2),
                     "VLR_TOTAL_NF": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PS": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PS": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "FATOR_QTD_PS": Numeric(10,4),
                     "FATOR_QTD_PSK": Numeric(10,4),
                     "FATOR_BONIFICADO_PS": Numeric(10,4),
                     "FATOR_BONIFICADO_PSK": Numeric(10,4),
                     "DT_INS": DateTime(),
                     "FLG_ORIGEM": Integer()})

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (0) (SQLEndTran)')
(Background on this error at: http://sqlalche.me/e/e3q8)

In [5]:
def get_data(dataproject, datasetprojectbi, table, clausule = "", columns = '*'):
    query = f"SELECT {columns} FROM `{dataproject}.{datasetprojectbi}.{table}` {clausule}"
    datatable = pd.read_gbq(
            query,
            project_id=dataproject,
            dialect="standard")
    return datatable

In [52]:
dfdate = get_data(dataproject = "bigqueryunilever", datasetprojectbi = "infrashop_clean", table = "Tabela_Calculo_6_Fato", columns = 'DISTINCT CAST(DT_INS AS DATE)')

dfdate['NaT'] = dfdate['f0_'].apply(lambda x: True if type(x) == pd._libs.tslibs.nattype.NaTType else False)

dfdate = dfdate[dfdate['NaT']!= True].drop('NaT', axis =1).sort_values(by=['f0_'])


Downloading: 100%|██████████| 33/33 [00:01<00:00, 25.00rows/s]


In [127]:
engine.execute(f"DELETE FROM FAT_PEDIDO_FATURADO_TESTE WHERE FLG_ORIGEM=3 and CAST(DT_INS AS DATE) = '2020-03-14'").fetchall

<bound method ResultProxy.fetchall of <sqlalchemy.engine.result.ResultProxy object at 0x7f1b16ec4410>>

In [124]:
for date in dfdate['f0_']:
    date = str(date).replace(' 00:00:00','')
    print(f"lendo FAT_PEDIDO_FATURADO BigQUery: {date}")
    df1 = get_data("bigqueryunilever", "infrashop_clean", "Tabela_Calculo_6_Fato", clausule = f"Where CAST(DT_INS AS DATE) = '{date}'")
    df1[columns] = df1[columns].fillna(-2)
    df1["DT_INS"] = df1["DT_INS"].astype('datetime64[ns]')
    print(f"exluindo FAT_PEDIDO_FATURADO: {date} do servidor")
    engine.execute(f"DELETE FROM FAT_PEDIDO_FATURADO_TESTE WHERE FLG_ORIGEM=3 and CAST(DT_INS AS DATE) = '{date}'")
    print(f"escrevendo FAT_PEDIDO_FATURADO: {date} no servidor")
    columns=['SK_PEDIDO', 'SK_FRONT', 'SK_PRODUTO', 'SK_KIT', 'SK_DATA', 'SK_CANAL_VENDA',
             'SK_ENDERECO', 'SK_CLIENTE', 'SK_DISTRIBUIDOR', 'SK_TIPO_PAGAMENTO', 'CD_NOTA_FISCAL',
             'QTD_ITEM_NF', 'VLR_ITEM_NF', 'VLR_TOTAL_NF', 'QTD_ITEM_A_ITEM_PS', 'QTD_ITEM_A_ITEM_PSK',
             'VLR_ITEM_A_ITEM_PS', 'VLR_ITEM_A_ITEM_PSK', 'FATOR_QTD_PS', 'FATOR_QTD_PSK',
             'FATOR_BONIFICADO_PS', 'FATOR_BONIFICADO_PSK', 'FLG_ORIGEM']
    engine.fast_executemany = True
    df1.to_sql('FAT_PEDIDO_FATURADO_TESTE',con=engine,index=False,if_exists="append",schema="dbo",
           dtype={   "CD_NOTA_FISCAL": Integer(),
                     "QTD_ITEM_NF": Integer(),
                     "VLR_ITEM_NF": Numeric(10,2),
                     "VLR_TOTAL_NF": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PS": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PS": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "FATOR_QTD_PS": Numeric(10,4),
                     "FATOR_QTD_PSK": Numeric(10,4),
                     "FATOR_BONIFICADO_PS": Numeric(10,4),
                     "FATOR_BONIFICADO_PSK": Numeric(10,4),
                     "DT_INS": DateTime(),
                     "FLG_ORIGEM": Integer()})


lendo FAT_PEDIDO_FATURADO BigQUery: 2020-01-27



Downloading: 100%|██████████| 4558/4558 [00:02<00:00, 1707.92rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-01-27 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-01-27 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-01-28



Downloading: 100%|██████████| 3376/3376 [00:02<00:00, 1415.97rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-01-28 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-01-28 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-01-29


...

exluindo FAT_PEDIDO_FATURADO: 2020-03-12 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-12 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-03-13



Downloading: 100%|██████████| 2331/2331 [00:01<00:00, 1248.63rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-03-13 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-13 no servidor
lendo FAT_PEDIDO_FATURADO BigQUery: 2020-03-14



Downloading: 100%|██████████| 13243/13243 [00:04<00:00, 2830.53rows/s]


exluindo FAT_PEDIDO_FATURADO: 2020-03-14 do servidor
escrevendo FAT_PEDIDO_FATURADO: 2020-03-14 no servidor


In [64]:
df1 = get_data("bigqueryunilever", "infrashop_clean", "Tabela_Calculo_6_Fato", clausule = "")


Downloading: 100%|██████████| 336682/336682 [01:08<00:00, 4905.49rows/s]


In [78]:
@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True

In [119]:
df1 = get_data("bigqueryunilever", "infrashop_clean", "Tabela_Calculo_6_Fato", clausule = f"Where CAST(DT_INS AS DATE) = '{date}'")


Downloading: 100%|██████████| 4558/4558 [00:02<00:00, 1782.35rows/s]


In [120]:
df1.head()

SK_PEDIDO  SK_FRONT  SK_PRODUTO  SK_KIT  SK_DATA  SK_CANAL_VENDA  \
0    2074112         1        9313      -2       -2              -2   
1    2074112         1       14261      -2       -2              -2   
2    2074112         1       13298      -2       -2              -2   
3    2074112         1       15790      -2       -2              -2   
4    2074112         1        7176      -2       -2              -2   

   SK_ENDERECO  SK_CLIENTE  SK_DISTRIBUIDOR  SK_TIPO_PAGAMENTO  ...  \
0           -2      195676               27                 -2  ...   
1           -2      195676               27                 -2  ...   
2           -2      195676               27                 -2  ...   
3           -2      195676               27                 -2  ...   
4           -2      195676               27                 -2  ...   

   QTD_ITEM_A_ITEM_PS  QTD_ITEM_A_ITEM_PSK  VLR_ITEM_A_ITEM_PS  \
0                12.0                 12.0               58.08   
1                16.0                 16.0               92.48   
2                 1.0                  1.0               13.11   
3                 4.0                  4.0               32.80   
4                 6.0                  6.0               25.92   

   VLR_ITEM_A_ITEM_PSK  FATOR_QTD_PS  FATOR_QTD_PSK  FATOR_BONIFICADO_PS  \
0                58.08             0              0                    0   
1                92.48             0              0                    0   
2                13.11             0              0                    0   
3                32.80             0              0                    0   
4                25.92             0              0                    0   

   FATOR_BONIFICADO_PSK                    DT_INS  FLG_ORIGEM  
0                     0 2020-01-27 14:21:19+00:00           3  
1                     0 2020-01-27 14:21:19+00:00           3  
2                     0 2020-01-27 14:21:19+00:00           3  
3                     0 2020-01-27 14:21:19+00:00           3  
4                     0 2020-01-27 14:21:19+00:00           3  

[5 rows x 24 columns]

In [122]:
columns=['SK_PEDIDO', 'SK_FRONT', 'SK_PRODUTO', 'SK_KIT', 'SK_DATA', 'SK_CANAL_VENDA', 'SK_ENDERECO', 'SK_CLIENTE', 'SK_DISTRIBUIDOR', 'SK_TIPO_PAGAMENTO', 'CD_NOTA_FISCAL', 'QTD_ITEM_NF', 'VLR_ITEM_NF', 'VLR_TOTAL_NF', 'QTD_ITEM_A_ITEM_PS', 'QTD_ITEM_A_ITEM_PSK', 'VLR_ITEM_A_ITEM_PS', 'VLR_ITEM_A_ITEM_PSK', 'FATOR_QTD_PS', 'FATOR_QTD_PSK', 'FATOR_BONIFICADO_PS', 'FATOR_BONIFICADO_PSK', 'FLG_ORIGEM']
df1[columns] = df1[columns].fillna(-2)
df1["DT_INS"] = df1["DT_INS"].astype('datetime64[ns]')
engine.fast_executemany = True
df1.to_sql('FAT_PEDIDO_FATURADO_TESTE',con=engine,index=False,if_exists="append",schema="dbo",
           dtype={   "CD_NOTA_FISCAL": Integer(),
                     "QTD_ITEM_NF": Integer(),
                     "VLR_ITEM_NF": Numeric(10,2),
                     "VLR_TOTAL_NF": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PS": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PS": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "FATOR_QTD_PS": Numeric(10,4),
                     "FATOR_QTD_PSK": Numeric(10,4),
                     "FATOR_BONIFICADO_PS": Numeric(10,4),
                     "FATOR_BONIFICADO_PSK": Numeric(10,4),
                     "DT_INS": DateTime(),
                     "FLG_ORIGEM": Integer()})



In [ ]:
df2.to_sql('FAT_PEDIDO_FATURADO_TESTE3',con=engine,index=False,if_exists="replace",schema="dbo",
           dtype={   "CD_NOTA_FISCAL": Integer(),
                     "QTD_ITEM_NF": Integer(),
                     "VLR_ITEM_NF": Numeric(10,2),
                     "VLR_TOTAL_NF": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PS": Numeric(10,2),
                     "QTD_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PS": Numeric(10,2),
                     "VLR_ITEM_A_ITEM_PSK": Numeric(10,2),
                     "FATOR_QTD_PS": Numeric(10,4),
                     "FATOR_QTD_PSK": Numeric(10,4),
                     "FATOR_BONIFICADO_PS": Numeric(10,4),
                     "FATOR_BONIFICADO_PSK": Numeric(10,4),
                     "DT_INS": DateTime(),
                     "FLG_ORIGEM": Integer()})

In [25]:
df2 = df1.head()

In [9]:
engine.execute("DELETE FROM FAT_PEDIDO_FATURADO WHERE FLG_ORIGEM=3")

In [ ]:
dfdata = get_data(dataproject = "bigqueryunilever", datasetprojectbi = "infrashop_clean", table = "Tabela_Calculo_6_Fato", columns = DT_INS)